In [64]:
#### import random
from matplotlib import pyplot as plt
import random
import numpy as np
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima_model import ARMA
from scipy import stats

In [103]:
class Agent():
    '''
    The class defines the main characteristics of my agents
    They have some money and assets
    '''
    def __init__(self, money, model):
        self.money = 0
        self.asset = Asset() 
        self.num_of_assets_owned = 0

        '''
        Here I'm assigning agents to a Expectation Formation type randomly
        To keep things simple for now, agents will use the same decisions-making technique
        '''
        self.EP_type = 1
#         if random.random() <0.5:
#             self.EP_type = 1 #1 - Agents with limited knowledge and predictive capabilities
#         else:
#             self.EP_type = 0 #0 - Agents with limited knowledge but better predictive capabilities
           
        
        '''
        Keeping track of all transactions agents make every time step
        '''
        self.transaction_history = list()
        if self.keep_asset:
            self.decision_keep = 1

        if self.sell_asset:
            self.decision_sell = 1
        
        if self.buy_asset:
            self.decision_buy = 1
            

        self.model = model
        '''
        Keeping track of predicted price.
        '''
        self.predict_price = 0
        
    def step(self):
        '''
        Two things happening every timestep:
        1) Agents will earn money
        2) Asset will step forward in time also
        3) Agents will fall into either 2 types of decision-making behaviours, simple linear regression or autoregression
        '''
        self.earn_money(20)
        
        self.asset.step()

        '''
        If the agent falls into the limited knowledge and predictive capabilities type, 
        they use a simple linear regression to predict the next asset price
        '''
        if self.EP_type == 1:
            self.predict_regession()
            '''
            Then, if the predicted price equal actual asset price then they keep the asset, 
            if predicted price is less actual asset price they sell, 
            and if the predicted price is greater than asset price they sell
            '''
            if (self.predict_price[0][0] > self.asset.update_price) and (self.money > self.asset.update_price):
                self.buy_asset()
                self.transaction_history.append('buy')
                
            elif self.predict_price[0][0] == self.asset.update_price:
                self.keep_asset()
                self.transaction_history.append('keep')

            elif self.predict_price[0][0] < self.asset.update_price:
                self.sell_asset()
                self.transaction_history.append('sell')
            
            print('Price Prediction = {0}, Market Asset Price = {1}, Decision = {2}, My Money = {3}'.format(self.predict_price, self.asset.update_price, self.transaction_history, self.money))
        else:
            pass
            #self.predict_AR()     
    
    def earn_money(self, amount):
        '''
        To introduce a possility of not earning disposable income every timestep, I included a probability
        '''
        if random.random( ) <0.2:
            self.money += amount
        
            
    def predict_regession(self):
        '''
        Expectation formation behaviour type 1:
        Simple linear regession predict the asset price considering previous asset prices. 
        Following each timestep, actual asset prices are added to the price history. 
        '''
        y=self.asset.price_history.reshape(-1, 1)
        X=self.asset.time_history.reshape(-1, 1)
        #print(y, X, y.shape, X.shape)
        reg=LinearRegression().fit(X,y)
        next_time = np.zeros((1, 1))
        next_time[0][0] = self.model.time + 1
        self.predict_price = reg.predict(next_time)
        
    def predict_AR(self):
        '''
        Expectation formation behaviour type 2:
        Autoregessive predictor
        '''
        pass
        #log_assetprice = np.log(self.asset.price_history)
        #log_assetprice_shift = log_assetprice - log_assetprice.shift()
        #get_stationarity(log_assetprice_shift)
        
        #plt.plot(log_assetprice)
        #model = ARIMA(log_assetprice, order=(2,1,2))
        #results = model.fit(disp=-1)
        #plt.plot(df_log_shift)
        #plt.plot(results.fittedvalues, color='red')
        
    def keep_asset(self):
        self.num_of_assets_owned += 0 
        self.money += 0
    
    def sell_asset(self):
        self.num_of_assets_owned -= 1 
        self.money += self.asset.price
    
    def buy_asset(self):
        self.num_of_assets_owned += 1
        self.money -= self.asset.price
        

In [104]:
class Asset():
    
    def __init__(self):
        '''
        Contains, asset price, a time log and their histories. 
        backdated prices that are randomly generated for a previous time periods.
        '''
        self.price = 50
        self.time = 1
        self.backdated = (50, 49)
        

        for i in range(len(self.backdated)):
            self.price_history = list(self.backdated)
        
        self.price_history = np.array(self.price_history)
        self.price_history = self.price_history.reshape(-1, 1)
        
        
        self.backdated_time = (0, 1)
        
        for i in range(len(self.backdated_time)):
            self.time_history = list(self.backdated_time)
        
        self.time_history = np.array(self.time_history)
        self.time_history = self.time_history.reshape(-1, 1)
        
    def step(self):
        '''
        Every time step, the price will update, keeping a log of time.
        '''
        self.update_price = stats.norm.rvs(loc = 50, scale = 1)
        self.time += 1
        
#         self.price_history.append(self.price)
        self.price_history = np.append(self.price_history, self.price)
#         self.time_history.append(self.time)
        self.time_history = np.append(self.time_history, self.time)
#         print(self.price_history, self.time_history)
        

In [105]:
class Model():
    '''
    This class includes functions suchas popuation, model iterations, and a step and run function
    '''
    def __init__(self,population, num_iterations):
        self.population = population
        '''
        Renaming the Asset, 'Produce' and associate my agents to it. 
        '''
        self.produce = Asset()
        self.agents = [Agent(0, self) for i in range (self.population)]

        self.num_iterations = num_iterations
        
        '''
        Recording time
        '''
        self.time = 0
        
    def step(self):
        '''
        Steps model forwards
        '''
        # Step Asset price
        self.produce.step()

        # Step agents 
        for i, agent in enumerate(self.agents):
            print('Agent: {0}/{1}'.format(i, self.population))
            agent.step()
        
        # Step time
        self.time += 1
    
  
    def run(self):
        '''
        For every time period, and for every agent in my list of agents, run the step function
        '''
        for t in range(self.num_iterations):
            print('Timestep: {}'.format(self.time))
            self.step()

In [106]:
# if __name__ == "__main__":
model = Model(population =1, num_iterations=6)
model.run()

Timestep: 0
Agent: 0/1
Price Prediction = [[49.66666667]], Market Asset Price = 49.932937478734, Decision = ['sell'], My Money = 50
Timestep: 1
Agent: 0/1
Price Prediction = [[49.8]], Market Asset Price = 50.65307408320846, Decision = ['sell', 'sell'], My Money = 100
Timestep: 2
Agent: 0/1
Price Prediction = [[49.9]], Market Asset Price = 50.20623091957139, Decision = ['sell', 'sell', 'sell'], My Money = 150
Timestep: 3
Agent: 0/1
Price Prediction = [[49.96190476]], Market Asset Price = 50.408585509308786, Decision = ['sell', 'sell', 'sell', 'sell'], My Money = 200
Timestep: 4
Agent: 0/1
Price Prediction = [[50.]], Market Asset Price = 49.49669066557758, Decision = ['sell', 'sell', 'sell', 'sell', 'buy'], My Money = 150
Timestep: 5
Agent: 0/1
Price Prediction = [[50.02380952]], Market Asset Price = 48.873024983034036, Decision = ['sell', 'sell', 'sell', 'sell', 'buy', 'buy'], My Money = 100


13/05: Meeting Notes
How to present model in dissertation:
UML
Flow diagrams + extra text, what each box does 
Diagram of ABM - Patricia Turner 
Pseudo code - for difficult parts 

Zero intelligent agents trade, condition: stay within budget, just having that creates real market price pattern
Randomly assign agents some random noise
Draw from normal distribution, log returns 
Adding guassian price changes
Starting from simple theory, doing in real life and proposing how to do things in ABMs
Link to homogeneous case, this is what existing theor and suggestions say. prove that heterogeneous is worth doing!
build a story from what everyone knows, to what I propose
Have simple trading stategy: when money = 0, agents can die. 
Create a document that wil help understand future goal
